In [ ]:
from IPython.utils import io
import sqlite3
import os
from itertools import combinations 

from IPython.core.debugger import set_trace

In [ ]:
conn = sqlite3.connect('got_treedata.db') #SQL database 
c=conn.cursor()



In [ ]:
def getdemostr(name,dbcursor,genderflag,unkcounter):
    #Returns a formatted string appropriate for family tree output.  Includes gender and birthdate/deathdate if available
    #name and dbcursor are straight-forward
    #genderflag represents the previously read gender.  Used only for gender of "Unknown" individuals (ugly -- need to fix)
    #unkcounter represents a counter on unknown parents -- needed to distinguish between multiple "Unknowns" in a family tree
    querystrdemo="SELECT Gender, YOB, YOD FROM PEOPLE WHERE Name=?" 
    GenStr=['F','M']
    
    c.execute(querystrdemo,(name,))
    curdemo=dbcursor.fetchall()
    compstr = name + ' ('
    if not curdemo:        
        unkcounter+=1
        #Unknowns require identification to avoid erroneous tree node generation.
        compstr += 'id=' + str(unkcounter) + ', '
        curdemo=[[(genderflag+1)%2,'','']]
    genderflag, bday, dday = curdemo[0]
    compstr+=GenStr[genderflag]
    if bday:
        compstr += ', birthday=' + str(bday)
    if dday:
        compstr += ', deathday=' + str(dday)
    compstr+=')'
    return compstr, genderflag, unkcounter

In [ ]:
def createHouseTree(housename,dbcursor):
    #construct and write the textual form of a family tree for the house designated by "housename"
    
    #Parentage tuples defining parent-and-child are considered valid if either one is in the given house.
    querystrbase="""SELECT * FROM PARENTAGE WHERE Parent IN (SELECT Name FROM PEOPLE WHERE House=?)
                UNION
                SELECT * FROM PARENTAGE WHERE Child IN (SELECT Name FROM PEOPLE WHERE House=?)"""    
    #We also include the other parent of a housemember's child in the tree (see README for explanation)
    querystraltpar="SELECT * FROM PARENTAGE WHERE Child=?"
    dbcursor.execute(querystrbase, (housename,housename))
    parentagebase = dbcursor.fetchall()
    if parentagebase:        
        unchildren=set([pc[1] for pc in parentagebase])
        for child in unchildren:       
            dbcursor.execute(querystraltpar,(child,))        
            addparentage=c.fetchall()
            for ap in addparentage:
                if ap not in parentagebase:
                    parentagebase.append(ap)
        
        partuples = parentagebase.copy()
        allchildren = [pc[1] for pc in partuples]
        unchildren = set(allchildren)
        child2p=dict()
        for child in unchildren:
            curparents = [pc[0] for pc in partuples if pc[1] == child]
            if len(curparents)<2:
                child2p[child]=set([curparents[0],'Unknown'])
            else:
                child2p[child]=set(curparents)
        allchildren=list(child2p.keys())
        allparents=set()
        for child in child2p:
            allparents=allparents.union(child2p[child])
        rdypeople= set(ap for ap in allparents if ap not in allchildren)
        genderflag=0
        unkcounter=0
        with open('HouseTrees/'+housename.replace(" ", "")+'.txt', 'w') as fwr: 
            while child2p:
                allchildren=list(child2p.keys())
                for child in child2p:
                    if child2p[child].issubset(rdypeople):
                        break        
                curchildren= [ac for ac in allchildren if child2p[ac]==child2p[child]]
                curparents = [ap for ap in child2p[child]]
                if curparents[0]=='Unknown':
                    curparents[0],curparents[1] = curparents[1], curparents[0]                
                fwr.write('\n')
                newsubtreestr=''
                for cpi in curparents:
                    [modtreestr,genderflag,unkcounter]=getdemostr(cpi,c,genderflag,unkcounter)
                    newsubtreestr+=modtreestr+'\n'
                for cci in curchildren:
                    [modtreestr,genderflag,unkcounter]=getdemostr(cci,c,genderflag,unkcounter)
                    newsubtreestr+='\t'+modtreestr+'\n'
                    #fwr.write('\t' + cci + '\n')
                    del child2p[cci]
                    rdypeople.add(cci)
                #set_trace()
                fwr.write(newsubtreestr)
                                   

In [ ]:
if not os.path.isdir('HouseTrees'):
    os.mkdir('HouseTrees')
hnquery="SELECT DISTINCT House FROM PEOPLE"
c.execute(hnquery)
rows = c.fetchall()
for row in rows:
    if 'House' in row[0]:
        createHouseTree(row[0],c)
        
conn.close()

In [ ]:

for file in os.listdir("HouseTrees"):
    if file.endswith(".txt"):
        curtreefile=os.path.join("HouseTrees/", file)        
        curpsfile = curtreefile[:-3]+'ps'
        with io.capture_output() as captured:
            %run familytreemaker $curtreefile
        %store captured.stdout > curtree.dot
        !dot -Tps2 curtree.dot -o $curpsfile | ps2pdf $curpsfile
        os.remove(curtreefile)
        os.remove(curpsfile)
        os.remove('curtree.dot')